# VISUALISATIONS NOTEBOOK

This notebook is created to draft some of the final visualisations.

In [1]:
import pandas as pd
import os
import shutil
import zipfile
import signal
import json
import numpy as np
import osmnx as ox
from pyproj import Transformer
from shapely.geometry import Polygon, LineString
from shapely.wkt import loads
from geopy.distance import geodesic
import geopandas as gpd
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
import matplotlib.cm as cm
import matplotlib.colors as mcolors  # Correct import for colors
from shapely.ops import linemerge
import folium
import math
from folium import plugins
import altair as alt

## Heat Map

In [2]:
def process_track(zone_output_path, track_id, edges_clean):

  track_path = os.path.join(zone_output_path, str(track_id) + '.json')

  # Load JSON data from the file
  with open(track_path, "r", encoding="utf-8") as file:
      data = json.load(file)

  # Extract relevant information safely
  waypoints_df = pd.DataFrame(data.get("waypoints"))
  coordinates_df = pd.DataFrame(data.get("coordinates"))

  # Obtain the edges information
  edges_df = coordinates_df[['source', 'target', 'CleanLongitude', 'CleanLatitude']].copy()
  edges_df['pair'] = edges_df.apply(lambda row: tuple(sorted([int(row['source']), int(row['target'])])), axis=1)

  # Group by 'pair' and aggregate to get count of coordinates
  result_df = edges_df.groupby('pair').agg(count_coordinates=('source', 'size')).reset_index()

  # Split the pair back into 'source' and 'target'
  result_df[['source', 'target']] = pd.DataFrame(result_df['pair'].tolist(), index=result_df.index)

  # Drop the 'pair' column as it's no longer needed
  result_df = result_df.drop(columns=['pair'])

  for index, row in result_df.iterrows():
    u = row['source']
    v = row['target']

    mask = ((edges_clean['u'] == u) & (edges_clean['v'] == v)) | ((edges_clean['u'] == v) & (edges_clean['v'] == u))
    current_list = edges_clean.loc[mask, 'list_tracks'].values[0]         # Check if track_id is not in the list for this edge

    # Only update count_tracks and list_tracks if track_id is not already in the list
    if track_id not in current_list:
        edges_clean.loc[mask, 'count_tracks'] += 1          # Update count_tracks
        edges_clean.loc[mask, 'list_tracks'] = edges_clean.loc[mask, 'list_tracks'].apply(lambda lst: lst + [track_id])           # Append track_id to list_tracks

  return edges_clean

In [3]:
def background_heat_map(edges_clean):

  edges_clean = edges_clean[edges_clean['count_tracks']>0].copy()

  # Reproject to a projected CRS, for example, UTM (EPSG: 32633 is UTM Zone 33N)
  edges_clean = edges_clean.to_crs(epsg=4326)

  # Compute the centroid for map centering
  center_lat = edges_clean['geometry'].centroid.y.mean()
  center_lon = edges_clean['geometry'].centroid.x.mean()

  # Create a folium map with a dark background
  m = folium.Map(location=[center_lat, center_lon], zoom_start=12, tiles="CartoDB Positron")

  # Define the color scale from yellow to red
  cmap = cm.copper_r
  norm = Normalize(vmin=edges_clean['count_tracks'].min(), vmax=edges_clean['count_tracks'].max())

  # Loop through the edges and plot them with a popup
  for _, row in edges_clean.iterrows():
      # Extract the line geometry and convert it to a list of coordinates
      coords = list(row['geometry'].coords)
      coords = [(lat, lon) for lon, lat in coords]  # Flip to (lat, lon)

      # Use the colormap to get the correct color
      color = cmap(norm(row['count_tracks']))
      color = mcolors.rgb2hex(color)  # Convert to Hex

      # Create a popup with count_tracks
      popup_text = f"{row['count_tracks']}"

      # Plot the polyline with a popup
      polyline = folium.PolyLine(
          locations=coords,
          color=color,
          weight=2.5,  # Default weight
          popup=folium.Popup(popup_text, max_width=300)
      ).add_to(m)

      # Add JavaScript for click event
      polyline.add_child(folium.Element(f"""
          <script>
          var poly = {polyline.get_name()};
          poly.on('click', function (e) {{
              this.setStyle({{weight: 4}});
          }});
          </script>
      """))

  return m

In [4]:
def obtain_zone_map(zone):

    print(f"Start the creation of the heatmap for {zone}.")

    # Load the shapefile
    nodes = gpd.read_file(f"../../Data/OSM-Data/{zone}/nodes.shp", encoding='utf-8')
    edges = gpd.read_file(f"../../Data/OSM-Data/{zone}/edges.shp", encoding='utf-8')

    # Read all needed dataframes
    discard_df = pd.read_csv(f"../../Data/Output/{zone}/discard_files.csv")
    output_df = pd.read_csv(f"../../Data/Output/{zone}/output_files.csv")
    no_osm_df = pd.read_csv(f"../../Data/Output/{zone}/no_osm_data.csv")

    edges_clean = edges[['u', 'v', 'geometry']].copy()

    # Create a new column that ensures (u, v) and (v, u) have the same representation
    edges_clean['pair'] = edges_clean.apply(lambda row: tuple(sorted([int(row['u']), int(row['v'])])), axis=1)
    edges_clean = edges_clean.drop_duplicates(subset=['pair']).drop(columns=['pair'])   # Drop duplicate pairs

    # Adding more columns
    edges_clean['count_tracks'] = 0
    edges_clean['list_tracks'] = [[] for _ in range(len(edges_clean))]

    list_tracks = output_df['track_id'].unique().tolist()
    total_to_process = len(list_tracks)

    i = 1
    zone_output_path = f"../../Data/Output/{zone}"
    for track in output_df['track_id'].unique().tolist():
        edges_clean = process_track(zone_output_path, track, edges_clean)
        print(f"\rProcessed {i} out of {total_to_process}", end='', flush=True)
        i += 1

    hm = background_heat_map(edges_clean)

    return hm

In [5]:
hm_canigo = obtain_zone_map('canigo')
hm_matagalls = obtain_zone_map('matagalls')
hm_vallferrera = obtain_zone_map('vallferrera')

Start the creation of the heatmap for canigo.
Processed 116 out of 116

C:\Users\ASUS\AppData\Local\Temp\ipykernel_68712\2731194232.py:9: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  center_lat = edges_clean['geometry'].centroid.y.mean()
C:\Users\ASUS\AppData\Local\Temp\ipykernel_68712\2731194232.py:10: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  center_lon = edges_clean['geometry'].centroid.x.mean()


Start the creation of the heatmap for matagalls.
Processed 7058 out of 7058

C:\Users\ASUS\AppData\Local\Temp\ipykernel_68712\2731194232.py:9: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  center_lat = edges_clean['geometry'].centroid.y.mean()
C:\Users\ASUS\AppData\Local\Temp\ipykernel_68712\2731194232.py:10: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  center_lon = edges_clean['geometry'].centroid.x.mean()


Start the creation of the heatmap for vallferrera.
Processed 1550 out of 1550

C:\Users\ASUS\AppData\Local\Temp\ipykernel_68712\2731194232.py:9: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  center_lat = edges_clean['geometry'].centroid.y.mean()
C:\Users\ASUS\AppData\Local\Temp\ipykernel_68712\2731194232.py:10: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  center_lon = edges_clean['geometry'].centroid.x.mean()


In [9]:
hm_vallferrera